# Classificação supervisionada (cobertura do solo) em imagem do Sentinel 1

Esse notebook utiliza o Google Earth Engine para realizar a classificação supervisionada de cobertura do solo em uma imagem do satélite Sentinel 1, todos os dados e ferramentas para a realização da classificação estão disponíveis na base de dados do Earth Engine, inclusive os algoritmos de classificação.

Outra vantagem é que os dados do Sentinel 1 já são disponibilizados com todo o pré processamento para determinação de intensidade de retorno feito, de acordo com os passos implementados na Toolbox do Sentinel 1, elaborada pela ESA.

Além das imagens do Sentinel 1, foi adquirido, também do Earth Engine, a base de dados de classificação da cobertura do solo CGLS, da Copernicus, que é utilizada neste notebook para treinar o modelo de classificação.

Dados do Sentinel 1: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD

Dados de cobertura do solo (CGLS): https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global


In [143]:
import ee
import geopandas as gpd
import json
import geemap

In [144]:
# Autenticar e inicializar o acesso ao Google Earth Engine
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AY0e-g6YomeqGRnA0MhKprqaA19IuxEm8qhejoi6rn1k9uYB8iza131yzr8

Successfully saved authorization token.


#### Os pacotes geopandas e json foram utilizados para carregar o shapefile da área de interesse e extrair suas coordenadas.

In [145]:
area = gpd.read_file('area_desafio/area.shp')
area_json = json.loads(area.to_json())

coords = area_json['features'][0]['geometry']['coordinates']
print(coords)

[[[-40.43448641364819, -2.9753515441904597], [-39.565388281268724, -2.9753515441904597], [-39.565388281268724, -3.7605736761144613], [-40.43448641364819, -3.7605736761144613], [-40.43448641364819, -2.9753515441904597]]]


#### A partir das coordenadas é possível construir um polígono da área de interesse, para recortar as imagens que serão utilizadas.

In [146]:
aoi = ee.Geometry.Polygon(coords)

#### Inicializando o mapa para exibição dos outputs.

In [147]:
Map = geemap.Map()

#### Carregando e filtrando dados do Sentinel 1

Foram carregados os dados do ano de 2019, por ser o mesmo ano dos dados de classificação de cobertura do solo, foram escolhidas as bandas VV e VH, e apenas imagens adquiridas com órbita descendente, por ser o único tipo de passagem disponível para a área de interesse.

A coleção de imagens foi reduzida a apenas uma imagem, a partir da seleção de valores medianos para as duas bandas.

In [148]:
s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(ee.Date('2019-01-01'), ee.Date('2019-12-31'))

s1_desc = s1_collection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));

s1_vvvh = s1_desc.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW'));

s1_composite = ee.Image.cat([s1_vvvh.select('VV').median(),s1_vvvh.select('VH').median(),]).clip(aoi)

#### Criando composto RGB da imagem

Para melhor visualização no mapa, a imagem do Sentinel 1 tratada de forma a criar bandas RGB.

Em seguida, a projeção foi mudada para Pseudo-Mercator, para permitir a amostragem de pontos.

rgb = ee.Image.rgb(s1_composite.select('VV'),
                   s1_composite.select('VH'),
                   s1_composite.select('VV').divide(s1_composite.select('VH')))

proj = ee.Projection('EPSG:3857');
rgb_r = rgb.reproject(proj, scale = 100)

#### Adicionando camada do Sentinel 1 (em RGB) ao mapa

In [149]:
Map.centerObject(aoi, 8)
Map.addLayer(rgb_r, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, "Sentinel 1")

#### Carregando dados de cobertura do solo da Copernicus, e visualizando os valores padrão das classes

In [150]:
lc_classes_bruto = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification').clip(aoi)

val_classes_bruto = lc_classes_bruto.get('discrete_classification_class_values').getInfo()
print(val_classes_bruto)

[0, 20, 30, 40, 50, 60, 70, 80, 90, 100, 111, 112, 113, 114, 115, 116, 121, 122, 123, 124, 125, 126, 200]


#### Atribuindo valores inteiros consecutivos às classes de cobertura do solo

In [151]:
n_classes = len(val_classes_bruto)
new_class_values = list(range(0, n_classes))
print(new_class_values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]


#### Visualizando valores da paleta de cores

In [152]:
class_palette = lc_classes_bruto.get('discrete_classification_class_palette').getInfo()
print(class_palette)

['282828', 'FFBB22', 'FFFF4C', 'F096FF', 'FA0000', 'B4B4B4', 'F0F0F0', '0032C8', '0096A0', 'FAE6A0', '58481F', '009900', '70663E', '00CC00', '4E751F', '007800', '666000', '8DB400', '8D7400', 'A0DC00', '929900', '648C00', '000080']


#### Remapeando dados da cobertura do solo com os novos valores de classes

In [153]:
lc_classes = lc_classes_bruto.remap(val_classes_bruto, new_class_values).select(['remapped'], ['discrete_classification'])
lc_classes = lc_classes.set('discrete_classification_class_values', new_class_values)
lc_classes = lc_classes.set('discrete_classification_class_palette', class_palette)

#### Gerando pontos em posições aleatórias, para classificação, e adicionando-os ao mapa

In [154]:
points = lc_classes.sample(**{
    'region': rgb_r.geometry(),
    'scale': 100,
    'numPixels': 10000,
    'seed': 0,
    'geometries': True
})

Map.addLayer(points, {}, 'training', False)

#### Amostrando a imagem do Sentinel 1 nos pontos gerados anteriormente, e separando-os em dados de treino (80%) e validação (20%)

In [155]:
# Bandas usadas para previsão.
bands = rgb_r.bandNames().getInfo()

# Valores das labels de cobertura do solo.
label = 'discrete_classification'

# Colocando os pontos sobre a imagem, para obter dados de treino do modelo.
sample = rgb_r.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 100
})

# Criando coluna de números aleatórios para separar dados em dados de treino e validação. 
sample = sample.randomColumn()

split = 0.8

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

#### Treinando o modelo de classificação e o aplicando sobre a imagem do Sentinel 1 inteira  

In [156]:
# Treinando o modelo.
classifier = ee.Classifier.smileRandomForest(32).train(training, label, bands)

# Classificando a imagem.
result = rgb_r.select(bands).classify(classifier)

#### Adicionando cores originais da base de dados de cobertura do solo aos resultados do modelo

In [157]:
class_values = lc_classes.get('discrete_classification_class_values').getInfo()
print(class_values)
class_palette = lc_classes.get('discrete_classification_class_palette').getInfo()
print(class_palette)

landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
['282828', 'FFBB22', 'FFFF4C', 'F096FF', 'FA0000', 'B4B4B4', 'F0F0F0', '0032C8', '0096A0', 'FAE6A0', '58481F', '009900', '70663E', '00CC00', '4E751F', '007800', '666000', '8DB400', '8D7400', 'A0DC00', '929900', '648C00', '000080']


#### Determinando acurácia global 

In [160]:
train_accuracy = classifier.confusionMatrix()
overall_acc = train_accuracy.accuracy().getInfo()
print('A acurácia global foi de: ', 100*overall_acc,''%')

A acurácia global foi de:  96.45059786028949 %


#### Determinando a acurácia do modelo quando executado sobre os dados de validação

In [161]:
validated = validation.classify(classifier)

test_accuracy = validated.errorMatrix('discrete_classification', 'classification')
val_acc = test_accuracy.accuracy().getInfo()
print('A acurácia de validação foi de: ', 100*val_acc,'%')

A acurácia de validação foi de:  50.75425790754258 %


#### Determinando o coeficiente Kappa, que diz o quão melhor a classificação foi do que uma atribuição aleatória, em escala de -1 (muito pior) a 1 (muito melhor)

In [162]:
print('Coeficiente Kappa:', test_accuracy.kappa().getInfo())

Coeficiente Kappa: 0.2496582319995872


In [163]:
Map.addLayer(landcover, {}, 'Land cover')
Map.add_legend(builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global')
Map

Map(center=[-3.3680064704094064, -39.99993734745807], controls=(WidgetControl(options=['position', 'transparen…